In [5]:
import os
import json
import pandas as pd
import traceback

In [6]:
from langchain.chat_models import ChatOpenAI

In [7]:
KEY = os.getenv("OPENAI_API_KEY")

In [8]:
llm = ChatOpenAI(openai_api_key = KEY, model='gpt-3.5-turbo', temperature=0.5)

e:\Gen AI - YT - Sunny\mcq_generator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [11]:
TEMPLATE = """
Text = {text}
You are an expert MCQ Maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure that the questions are not repeated and check all the questions to be confirming the text as well.
Make sure you format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
#### RESPONSE_JSON
{response_json}

"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template = TEMPLATE
)

In [13]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz',verbose=True)

In [14]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [15]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['subject','quiz'], 
    template=TEMPLATE
)

In [16]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key='review', verbose=True)

In [17]:
generate_evaluate_chain = SequentialChain(
    chains = [quiz_chain,review_chain],
    input_variables=['text','number','subject','tone','response_json'],
    output_variables=['quiz','review'],
    verbose=True
)

In [18]:
file_path = 'E:\Gen AI - YT - Sunny\mcq_generator\data.txt'

In [19]:
with open(file_path, 'r') as f:
    TEXT = f.read()

In [20]:
print(TEXT)

In physics, a force is a push or pull between objects. It is called an interaction because if one object acts on another, its action is matched by a reaction from the other object.[1] This idea is known as Newton's third law,[2] where action and reaction are "equal and opposite" [3] (matched). The objects are just the things the force acts between. Different forces act between different sorts of object. For example, gravity acts between objects with mass, like the sun and the earth. Another example is electromagnetic force, which acts between objects with charge, like an electron and the nucleus of an atom. Gravity and electromagnetic force are two examples of forces.

A force changes the state of an object (some physical quantity changes) or, strictly, the states of two objects, since the force is an interaction. For example, a force causes an affected object to be pushed or pulled in a certain direction. This changes the object's momentum. Forces cause objects to accelerate, add to t

In [21]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [22]:
NUMBER = 5
SUBJECT = 'Biology'
TONE = 'simple'

In [23]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
        'text':TEXT,
        'number':NUMBER,
        'subject':SUBJECT,
        'tone':TONE,
        'response_json':json.dumps(RESPONSE_JSON)
        }
    )

e:\Gen AI - YT - Sunny\mcq_generator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text = In physics, a force is a push or pull between objects. It is called an interaction because if one object acts on another, its action is matched by a reaction from the other object.[1] This idea is known as Newton's third law,[2] where action and reaction are "equal and opposite" [3] (matched). The objects are just the things the force acts between. Different forces act between different sorts of object. For example, gravity acts between objects with mass, like the sun and the earth. Another example is electromagnetic force, which acts between objects with charge, like an electron and the nucleus of an atom. Gravity and electromagnetic force are two examples of forces.

A force changes the state of an object (some physical quantity changes) or, strictly, the states of two objects, since the force is an interaction. For example, a force causes an affected object to be pushed or p

In [24]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1728
Prompt Tokens:1090
Completion Tokens:638
Total Cost:0.0029110000000000004


In [25]:
response

{'text': 'In physics, a force is a push or pull between objects. It is called an interaction because if one object acts on another, its action is matched by a reaction from the other object.[1] This idea is known as Newton\'s third law,[2] where action and reaction are "equal and opposite" [3] (matched). The objects are just the things the force acts between. Different forces act between different sorts of object. For example, gravity acts between objects with mass, like the sun and the earth. Another example is electromagnetic force, which acts between objects with charge, like an electron and the nucleus of an atom. Gravity and electromagnetic force are two examples of forces.\n\nA force changes the state of an object (some physical quantity changes) or, strictly, the states of two objects, since the force is an interaction. For example, a force causes an affected object to be pushed or pulled in a certain direction. This changes the object\'s momentum. Forces cause objects to accele

In [26]:
quiz = response['quiz']

In [27]:
quiz = json.loads(quiz)

In [28]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [29]:
quiz_table_data

[{'MCQ': 'What is a force in physics?',
  'Choices': 'a: A push or pull between objects | b: A type of energy | c: A form of matter | d: A type of motion',
  'Correct': 'a'},
 {'MCQ': 'Which law states that for every action, there is an equal and opposite reaction?',
  'Choices': "a: Newton's first law | b: Newton's second law | c: Newton's third law | d: Newton's fourth law",
  'Correct': 'c'},
 {'MCQ': 'What is the unit of measurement for the strength of a force?',
  'Choices': 'a: Kilograms | b: Meters | c: Newtons | d: Watts',
  'Correct': 'c'},
 {'MCQ': 'Which force acts between objects with mass?',
  'Choices': 'a: Electromagnetic force | b: Gravity | c: Friction | d: Tension',
  'Correct': 'b'},
 {'MCQ': 'How does a force affect objects?',
  'Choices': 'a: By changing their color | b: By changing their taste | c: By changing their motion or shape | d: By changing their temperature',
  'Correct': 'c'}]

In [30]:
quiz = pd.DataFrame(quiz_table_data)

In [31]:
quiz.to_csv('Biology_quiz', index=False)

In [32]:
quiz

,MCQ,Choices,Correct
0,What is a force in physics?,a: A push or pull between objects | b: A type ...,a
1,"Which law states that for every action, there ...",a: Newton's first law | b: Newton's second law...,c
2,What is the unit of measurement for the streng...,a: Kilograms | b: Meters | c: Newtons | d: Watts,c
3,Which force acts between objects with mass?,a: Electromagnetic force | b: Gravity | c: Fri...,b
4,How does a force affect objects?,a: By changing their color | b: By changing th...,c
